# Implementação KEM do NTRU-Prime

## Parameters

In [1]:
def verifyW(p, q, w, indice):
    while (2*p < 3*w):
        indice = indice + 1
        w = p//indice
    
    while (q < (16*w + 1)):
        indice = indice + 1
        w = p//indice

    return w

In [2]:
p = next_prime(120)
q = next_prime(120)
w = p//4

w = verifyW(p,q,w,4)


RingQ.<x> = PolynomialRing(GF(q),'x')
if (x^p-x-1).is_irreducible() is False:
    print("Error\n")

Z = IntegerRing()
Ring  = PolynomialRing(Z,'x')
Ring3 = PolynomialRing(GF(3),'x')

R  = QuotientRing(Ring, Ring.ideal(x^p-x-1))
R3 = QuotientRing(Ring3, Ring3.ideal(x^p-x-1))
Rq = QuotientRing(RingQ, RingQ.ideal(x^p-x-1))

## Funções auxiliares

In [3]:
def roundEach(f,b=None):
    ff = list(f)
    if b == None:
        return ff
    else:
        fp = map(lift,[Mod(a,b) for a in ff])
        return [u if u <= b//2 else u-b for u in fp ]

def round3(f,b=None):
    ff = list(f)
    if b == None:
        return ff
    else:
        fp = map(lift,[Mod(a,b) for a in ff])
        return [u if u <= b//3 else u-b for u in fp ]

def toR(vec):
    return R(vec)

def computeH(f,g):
    _f = Rq(f)
    _g = Rq(g)
    
    try:
        hq = (_g/_f)
        return (True,Rq([lift(a) for a in list(hq)]))
    except:
        return (False,0)

In [4]:
import random as rn

def small(P=p):
    u = [rn.choice([-1,0,1]) for i in range(p)]
    return u

def smallW(P=p,W=w):
    u = [rn.choice([-1,1]) for i in range(w)] + [0]*(p-w)
    rn.shuffle(u)
    return u

In [5]:
def verifyG():
    while True:
        g = small()
        if(R3(g).is_unit()):
            break
            
    return g

In [6]:
def formatC(ciphertext):
    size = len(ciphertext)
    
    c = ciphertext[33:(size-1)].split(",")
    
    for i in range(len(c)):
        c[i] = int(c[i])
    
    return c

## Key generation, Encapsulate e Decapsulate

In [7]:
import hashlib

def KeyGen(j=q, k=p, l=w):
    G = verifyG()
    F = smallW()
    f = 3 * toR(F)
    g = toR(G)
    h = Rq(g)/Rq(f)
    _g = R3(g)^(-1)
    return {'f': R(F), 'g': _g, 'pk' : h}

def Encapsulate(pk):
    Rr = smallW()
    r = toR(Rr)
    c = roundEach(Rq(pk)*Rq(r), b=q)
    print r
    fhash = hashlib.sha512()
    fhash.update(str(Rr).encode('utf-8'))
    divisao = fhash.digest()
    C = divisao[:32]
    K = divisao[32:]
    c = str(c)
    
    return {'Cc': C+c, 'K': K}
    
def Decapsulate(ciphertext,f,g):
    C = ciphertext[:32]
    c = formatC(ciphertext)
    F = Rq(3*f)
    e = round3(F*Rq(c),b=q)
    print e
    l = R3(e)*q
    rLinha = roundEach(l,b=3)
    print rLinha
    if True:
        fhash = hashlib.sha512()
        fhash.update(str(rLinha).encode('utf-8'))
        divisao = fhash.digest()
        CLinha = divisao[:32]
        KLinha = divisao[32:]
    
    return base64.b64encode(KLinha)

## Test

In [8]:
import base64

def run():
    keys   = KeyGen()
    crypto = Encapsulate(keys['pk'])
    decryp = Decapsulate(crypto['Cc'],keys['f'],keys['g'])
    return base64.b64encode(crypto['K'])==decryp

In [9]:
run()

xbar^118 - xbar^83 + xbar^57 - xbar^50 - xbar^43 - xbar^24 + xbar^15
[-1, -2, -1, -2, -3, -2, 0, 6, 6, 4, -1, -5, -3, 1, 1, -5, -3, 2, 3, 0, -1, 3, 1, -6, -2, 4, 3, 1, 2, 0, -3, -1, -2, -2, -1, 3, 5, 6, 2, 2, 2, -1, -3, -2, 0, -3, -2, 4, 0, 1, 2, -2, 1, 2, 0, 1, 1, 1, 1, 3, -2, -2, -1, -1, 2, -1, -5, -3, -2, -1, 0, 5, -4, -2, 0, -1, -1, 2, 2, -2, -2, 4, 0, -1, -1, 0, 2, 0, -6, -2, -2, -4, -1, 1, 0, 2, -2, 0, 1, -2, -1, 0, 3, -1, 2, 0, -3, -2, 0, 2, 2, 2, -2, -2, 2, 0, -1, 1, -1, -1, 0, 4, 0, 4, 6, 1, 2]
[-1, 1, -1, 1, 0, 1, 0, 0, 0, 1, -1, 1, 0, 1, 1, 1, 0, -1, 0, 0, -1, 0, 1, 0, 1, 1, 0, 1, -1, 0, 0, -1, 1, 1, -1, 0, -1, 0, -1, -1, -1, -1, 0, 1, 0, 0, 1, 1, 0, 1, -1, 1, 1, -1, 0, 1, 1, 1, 1, 0, 1, 1, -1, -1, -1, -1, 1, 0, 1, -1, 0, -1, -1, 1, 0, -1, -1, -1, -1, 1, 1, 1, 0, -1, -1, 0, -1, 0, 0, 1, 1, -1, -1, 1, 0, -1, 1, 0, 1, 1, -1, 0, 0, -1, -1, 0, 0, 1, 0, -1, -1, -1, 1, 1, -1, 0, -1, 1, -1, -1, 0, 1, 0, 1, 0, 1, -1]


False